---
hide: true
toc: true
layout: notebook
title: Individual Review
description: Recap of key events and learnings
type: tangibles
courses: { csp: {week: 12} }
---

## Issue Recap
During the project, I have tried to keep issues to track work and progress.  

[Online Issues](https://github.com/Cosmic-Carnage/Issues/issues): 102 commits since October

- [Contributions](https://github.com/raymondYsheng)
- [Spacebook API Functions](https://github.com/Cosmic-Carnage/Issues/issues/24)
- [Spacebook API 500 Error](https://github.com/Cosmic-Carnage/Issues/issues/32)
- [Leaderboard API CORS Error](https://github.com/Cosmic-Carnage/Issues/issues/30)
- [DrawIO Diagram](https://github.com/Cosmic-Carnage/Issues/issues/27)

## Overview of Spacebook

### Main Spacebook file:
```java
public class Spacebook {
    
    @Id
    @GeneratedValue(strategy = GenerationType.AUTO)
    private Long id;
   
    public Long getId() {
      return id;
  }
  public void setId(Long id) {
      this.id = id;
  }

  private byte[] image;

    public byte[] getImage() {
        return image;
    }

    public void setImage(byte[] image) {
        this.image = image;
    }


    @Column
    private int like;
    @Column
    private int dislike; // track the number of dislikes

    public int getLike() {
        return like;
    }

    public void setLike(int like) {
        this.like = like;
    }

    public int getDislike() {
        return dislike;
    }

    public void setDislike(int dislike) {
        this.dislike = dislike;
    }

    public String toString() {
        return "Product [id=" + id + ", image=" + image + "]";
    }
}
```
Defines likes, dislikes, id, and the image variable as a byte array

### ApiController for deleting posts:
```java
    @DeleteMapping("/delete/{id}")
    public ResponseEntity<Spacebook> deleteSpacebook(@PathVariable long id) {
        Optional<Spacebook> optional = spacebookRepo.findById(id);
        if (optional.isPresent()) {
            Spacebook spacebook = optional.get();
            spacebookRepo.deleteById(id);
            return new ResponseEntity<>(spacebook, HttpStatus.OK);
        }
        return new ResponseEntity<>(HttpStatus.BAD_REQUEST);
    }
```

### Scrapped Comment Function to be implemented for the Future
```java
private List<String> comments = new ArrayList<>();
    private static List<String> initComment() {
        ArrayList<String> comments = new ArrayList<>();
        comments.add("You suck");
        return comments;
    };

@PostMapping("/comment/{id}")
public ResponseEntity<Spacebook> setComment(@PathVariable String comment, @PathVariable long id) {
    Optional<Spacebook> optional = repository.findById(id);
    if (optional.isPresent()) {
        Spacebook spacebook = optional.get();
    }

List<String> comments = spacebook.getComments();
comments.add(comment); 

spacebook.setComments(comments);
```

Meant to upload comments to an associated spacebook, scrapped because we needed to focus on integration

### Old Upload Code
```java
@PostMapping("/upload")
    public ResponseEntity<String> saveSpacebookToDB(@RequestParam("file") String file) {
        if (file.isEmpty()) {
            return new ResponseEntity<>("Please select a file to upload.", HttpStatus.BAD_REQUEST);
        }
    }
    try {
            byte[] imageBytes = Base64.getDecoder().decode(file);
            Spacebook spacebook = new Spacebook();
            spacebook.setImage(imageBytes);

            Spacebook savedSpacebook = spacebookRepo.save(spacebook);
            return new ResponseEntity<>("Image uploaded successfully. Spacebook ID: " + savedSpacebook.getId(), HttpStatus.OK);
        } catch (Exception e) {
            e.printStackTrace();
            return new ResponseEntity<>("Upload failed.", HttpStatus.INTERNAL_SERVER_ERROR);
        }
```

## Overview of Leaderboard
My idea to create a HashMap to store associated player names and scores. I felt like a HashMap would be more intuitive to use than a 2D array or related ArrayLists.
```java
public static HashMap<String, Integer> init() {
        HashMap<String, Integer> leaderboardHash = new HashMap<>();
        {
            leaderboardHash.put("Player1", 1000);
            leaderboardHash.put("Player2", 800);
            leaderboardHash.put("Player3", 1200);
            leaderboardHash.put("Player4", 4000);
            leaderboardHash.put("Player5", 670);
            leaderboardHash.put("Player6", 320);
            leaderboardHash.put("Player7", 570);
            leaderboardHash.put("Player8", 129);
            leaderboardHash.put("Player9", 250);
            leaderboardHash.put("Player10", 875);
        }
        return leaderboardHash;
    }
```


## Key Commits:
- [String to byte array](https://github.com/Cosmic-Carnage/cosmic_backend_final/commit/a7e13d8d1b692a0df8205d4165f2336b5412dfe9) This is where we decided to encode base64 strings
- [Create basic frontend for future implementation](https://github.com/Cosmic-Carnage/Project-CC/commit/683b42269d518d2895d59cff9e3db11de830f3fd) This is where I worked on making a page for frontend for future Spacebook integration


## Struggles
- 500 Error: When uploading using Spacebook, the new spring portfolio template threw an internal server error. I used the debug function in VSCode and helped revert to the old spring portfolio to resolve this.
- Main.java Errors: Small compiler errors that don't show up before running main were difficult to understand.

## Collegeboard Quiz Notes:
- Using scratch paper or help thought process
- Question 4: remember that int operations return ints not decimals
- Question 10: Recursive function, remember to look out for break errors
- Question 16: Use an example case to help figure out if it is +/- 1
- Question 18: Use Math.Random in personal code from now on for AP prep
- Question 19: Need to review boolean algebra, "not" "not" cancels them out

## Plans to study for AP Exam:
- Doing extra practice MC and FRQ on Collegeboard
- As AP Exams get closer, ramp up practice
    - Do full practice exams
- Review in class student teaching, discuss any problems
- Create programs and practical functions with topics unfamiliar:
    - Especially: Recursion and Inheritance (`super` and `extends`)

## Trimester 1 Reflections
### What did I learn?
- Familiarize with Spring portfolio
- Inheritance and Boolean Algebra
- public vs private in Java

### Positive Accomplishments
- I am much more familiar with backend/spring portfolio as well as Postman
- Spacebook likes, dislikes, upload assist
- Developed debugging knowledge (reading the error messages)

### What can I improve on?
- I want to learn more about boolean algebra and get better and faster at it
- 